In [9]:
WKDIR=/workspace/hraijc/Blueberry/Blueberry_trio/contigbin
VCAE=/output/genomic/fairGenomes/Plant/Vaccinium/corymbosum/var_na/sex_na/2x/assembly_w85_20/v1.3/VcaeV1.3.p0.fa
MERGEDIR=/workspace/hraijc/Blueberry/Blueberry_trio/Assembly/postcontamremoval/merged
LOG=${WKDIR}/log
NUIBAM=${WKDIR}/Nuih1h2_cleanMerge_to_Vcae13_filtered.bam
M7BAM=${WKDIR}/M7h1h2_cleanMerge_to_Vcae13_filtered.bam
NUILM=${WKDIR}/LinkageMap/simplex_Nui.csv
#NUILM=${WKDIR}/LinkageMap/Nui_test.csv
M7LM=${WKDIR}/LinkageMap/simplex_M7.csv

mkdir -p $LOG
cd $WKDIR

In [10]:
grep ">" $VCAE | head

>seq-0-001
>seq-0-002
>seq-0-003
>seq-0-004
>seq-0-005
>seq-0-006
>seq-0-007
>seq-0-008
>seq-0-009
>seq-0-010


In [11]:
module load samtools


In [78]:
CHROM=seq-0-001
POS=21582035






In [79]:
# Two contigs map to this postion in the reference
samtools view $NUIBAM ${CHROM}:${POS}-${POS} | awk '{print $1}'

h2tg000547l_1
h1tg000687l_1


In [80]:
# The genotype of the two contigs is indicated in the 5th column. the reference contig is in the 1st column and the reference position is in the 2nd column.
# The order of the genotypes is the same as the order spit out by samtools view (thankfully).
samtools mpileup -r "${CHROM}:${POS}-${POS}" $NUIBAM 2>/dev/null # | awk '{print $1,$5}'

seq-0-001	21582035	N	2	At	~~


In [81]:
# Confirm in Geneious. 
samtools view $NUIBAM ${CHROM}:${POS}-${POS} > test.sam

## Use PrintAtPositon.py to parse bam files to score contigs as reference or alternative alleles. 

In [11]:
ml conda
conda deactivate
module unload conda
module load pfr-python3
module load samtools


python3 /workspace/hraijc/Gitrepos/High-quality-genomes/Blueberry/Trio-binned\ Hifi\ M7xNui/ContigBinning/PrintAtPostion.py --help

usage: PrintAtPostion.py [-h] input_csv output_csv bam_file

Execute samtools commands using data from a CSV file.

positional arguments:
  input_csv   Input CSV file containing chromosomes and postions of interest.
  output_csv  Output CSV file to save the results.
  bam_file    Sorted and indexed BAM file.

optional arguments:
  -h, --help  show this help message and exit


In [12]:
sbatch << EOF
#!/bin/bash
#SBATCH -J samtools_python
#SBATCH -o ${LOG}/%J.out
#SBATCH -e ${LOG}/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=2:00:00

ml conda
conda deactivate
module unload conda
module load pfr-python3
module load samtools

python3 /workspace/hraijc/Gitrepos/High-quality-genomes/Blueberry/Trio-binned\ Hifi\ M7xNui/ContigBinning/PrintAtPostion.py $NUILM ${WKDIR}/LinkageMap/Nui_contig_ids.csv $NUIBAM
python3 /workspace/hraijc/Gitrepos/High-quality-genomes/Blueberry/Trio-binned\ Hifi\ M7xNui/ContigBinning/PrintAtPostion.py $M7LM ${WKDIR}/LinkageMap/M7_contig_ids.csv $M7BAM

EOF

Submitted batch job 2565971
